# `pidgin.specification.markdown` 

Controls the output behavior of the `pidgin.weave` step.  `pidgin.\` will `tangle` indented block code, footnotes, and reference links before rendering the __Markdown__ input as html output.

In [1]:
    if __name__ == '__main__': 
        %reload_ext pidgin

In [2]:
    import ast, jinja2, sys, nbconvert, nbformat, contextlib, collections, pygments, IPython, mistune, doctest, re, traitlets, textwrap, types, typing as t, importlib, importnb,  jinja2, builtins, jsonschema
    from pidgin import tangle, shell, formatter

In [3]:

```
import pidgin
try: import graphviz
except: graphviz = None
from pidgin.docs.references import *
```

`pidgin` adds typographic indicators to `mistune.Markdown` to include [`graphviz`][what is graphviz?] and [`doctest`]


In [4]:
    class Markdown(mistune.Renderer, mistune.Markdown, shell.Shell):
        run = traitlets.Bool(default_value=True)
        def __init__(Markdown, *args, **kwargs):
            shell.Shell.__init__(Markdown, *args, **kwargs)
            mistune.Renderer.__init__(Markdown)
            mistune.Markdown.__init__(Markdown, renderer=Markdown, block=BlockLexer, inline=InlineLexer)
            Markdown.block.shell = Markdown
            Markdown._codespan_buffer = []
            Markdown._block_code_buffer = []
            
        def __call__(self, text):
            from pidgin import transform_cell
            
            text = transform_cell.strip_blank_lines(text.rstrip('\n'))
            self.blob, self.linenos = text, tuple()
            return self.parse(text)

        render = __call__
        
        def __enter__(Markdown): 
            Markdown.run = False
            return Markdown
        
        def __exit__(Markdown, *e): Markdown.run = True
                    
        def pygment(Markdown, text, lang=None, **kwargs):
`Markdown.pygment` is a customizable utility to __highlight__ inline and block code.
            
            lang = lang or 'python'
            import pygments
            return pygments.highlight(text, pygments.lexers.get_lexer_by_name(lang, stripall=True), pygments.formatters.html.HtmlFormatter(noclasses=True, **kwargs))
        
        def codespan(Markdown, text, lang=None):
`Markdown.codespan` reformats html output to use inline elements: div->span & pre->code.
            
            Markdown._codespan_buffer.append(text)
            if not text.endswith('\\'):
                Markdown.run_literal('\n'.join(Markdown._codespan_buffer).rstrip())
                Markdown._codespan_buffer = []
            text = text.rstrip('\\')
            return F"""<span{
                Markdown.pygment(text, prestyles="display: inline-block; vertical-align: middle", nobackground=True).strip()[4:-4]
            }span>""".replace('<pre', '<code').replace('</pre', '</code')

        
        def block_code(Markdown, text, lang=None):
`Markdown.block_code` applies __pygments__ highlighting to code blocks with corresponding line numbers
to the parent code.
            
            if lang == "": 
                Markdown._block_code_buffer.append(text)
                if not text.endswith('\\'):
                    Markdown.run_literal('\n'.join(Markdown._block_code_buffer))
                    Markdown._block_code_buffer = []
            
            if hasattr(Markdown, 'blob'):
                m = next(re.finditer('\W+'.join(map(re.escape, filter(bool, map(str.strip, text.splitlines())))), Markdown.blob))
                Markdown.linenos += len(m.string[:m.span()[0]].splitlines()) + (lang is not None),
                Markdown.blob = m.string[m.span()[1]:]
                try:
                    return Markdown.pygment(
                        text.rstrip('\\'), lang, linenos='table', linenostart=sum(Markdown.linenos)
                    ).replace("""<td class="code">""", """<td style="text-align: left;" class="code">""")
                finally:
                    Markdown.linenos += len(m.string[slice(*m.span())].splitlines())-2,

            return Markdown.pygment(text.rstrip('\\'), lang)
        
        def run_literal(Markdown, str):
`Markdown.run_literal` is used by `Markdown.block_code` and `Markdown.codespan` to execute the contents.
            
            return Markdown.run and super().run_cell(textwrap.indent(str, ' '*4), silent=True)
        
        def output_doctest(self): return self.renderer.doctest(self.token['text'])
        
        def output_graphviz(self): return self.renderer.graphviz(self.token['text'])
        
        def doctest(self, text): return super().block_code(text)
        def graphviz(self, text): 
            return __import__('graphviz').Source(text)._repr_svg_()

        
        def header(self, text, level, raw=None):
            import html, slugify
            slug = html.escape(__import__('slugify').slugify(text, lowercase=False))
            return F"""<a href="#{slug}"><h{level} id="{slug}">{text}</h{level}></a>"""        

In [5]:
    class DocTestGrammar:
`DocTestGrammar` adds `doctest` to `pidgin`'s `mistune.BlockGrammar`.  The ellipsis flag is enabled by default.
    
        doctest = doctest.DocTestParser._EXAMPLE_RE
        
    class DoctestLexer:
`DoctestLexer` adds a `"doctest"` token types type to the parsed __Markdown__.        

        def parse_doctest(self, m): self.tokens.append({'type': 'doctest', 'text': m.string[slice(*m.span())]})
            

1,class DocTestGrammar:
4 5 6,doctest = doctest.DocTestParser._EXAMPLE_RE class DoctestLexer:
9,"def parse_doctest(self, m): self.tokens.append({'type': 'doctest', 'text': m.string[slice(*m.span())]})"


In [6]:
### Learn more about [`doctest`][]

In [7]:
    class GraphvizGrammar:
`GraphvizGrammar` recognizes [graphviz syntax] in the `mistune.BlockGrammar`.        

        graphviz = re.compile("^(di){0,1}graph\s*\{(\{(\{(\{(\{(\{.*?\}|.)*?\}|.)*?\}|.)*?\}|.)*?\}|.)*?\}", re.DOTALL)
    
    class GraphvizLexer:
        def parse_graphviz(self, m): 
            self.tokens.append({'type': 'graphviz', 'text': m.string[slice(*m.span())]})       

1,class GraphvizGrammar:
4 5 6 7 8,"graphviz = re.compile(""^(di){0,1}graph\s*\{(\{(\{(\{(\{(\{.*?\}|.)*?\}|.)*?\}|.)*?\}|.)*?\}|.)*?\}"", re.DOTALL) class GraphvizLexer: def parse_graphviz(self, m): self.tokens.append({'type': 'graphviz', 'text': m.string[slice(*m.span())]})"


In [8]:
> #### [what is graphviz?][]
>> Learn more about [graphviz][what is graphviz?] and its [syntax][graphviz syntax].

In [9]:
## Block and inline `mistune` lexers.

In [10]:
    class BlockLexer(GraphvizLexer, DoctestLexer, mistune.BlockLexer): 
        class grammar_class(GraphvizGrammar, DocTestGrammar, mistune.BlockGrammar): ...
        
        def __init__(BlockLexer, rules=None, **kwargs): super().__init__(rules, **kwargs)
        
        def parse_fences(self, m):
            super().parse_fences(m)
            self.tokens[-1]['lang'] = self.tokens[-1]['lang'] or ''
            
        def parse_block_quote(self, m):
            object = m.string[slice(*m.span())]
            if object not in self.shell.user_ns: self.shell.user_ns[object] = object
            super().parse_block_quote(m)
            
    for attr in "doctest graphviz".split():
        for key in "default_rules footnote_rules list_rules".split():[
            setattr(BlockLexer, key, list(BlockLexer.default_rules)), 
            getattr(BlockLexer, key).insert(getattr(BlockLexer, key).index('block_quote'), attr)]

In [11]:
    class InlineLexer(mistune.InlineLexer):
        def output_reflink(self, m):
            key = mistune._keyify(m.group(2) or m.group(1)).lower()
            if key in self.renderer.user_ns:
                m2 = mistune.BlockGrammar.def_links.match(F"[xxx]: {self.renderer.user_ns[key]}")
                ret = self.links[key] = dict(link=m2.group(2), title=m2.group(3))
                return self._process_link(m, ret['link'], ret['title'])
            return super().output_reflink(m)
        def output_emphasis(self, m):
            object = m.string[slice(*m.span())]
            if object not in self.renderer.user_ns:
                self.renderer.user_ns[object] = object
            text = m.group(2) or m.group(1)
            text = self.output(text)
            return self.renderer.emphasis(text)